# 4.6 Walkthrough Merging Instacart Data

### 1. Download the new data set
### 2. Unzip the file and move it into your project folder
### 3. Import the data set into Jupyter as the new dataframe df_ords_prior
### 4. Check the dimensions of the imported dataframe
### 5. Merge the dataframe with your df_ords dataframe
### 6. Use the indicator argument to check whether there was a full match between the two dataframes


In [1]:
#Steps 1 & 2 are done outside of Jupyter

### 3. Import the data set into Jupyter as the new dataframe df_ords_prior

In [1]:
#Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Create Path to import

In [4]:
path = r'/Users/bethanybreed/Documents/A Career Foundry/07-2022 Instacart Basket Analysis'

In [5]:
# Import most up to date orders_checked

In [6]:
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'prepared data', 'orders_checked.csv'), index_col = False)

In [7]:
# Import df_ords_prior

In [8]:
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'original data', 'order_products_prior.csv'), index_col = False)

### 4. Check the dimensions of the imported data frames

In [9]:
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2539329,1,1,2,8,median
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [10]:
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
df_ords.shape

(3421083, 7)

In [12]:
df_ords_prior.shape

(32434489, 4)

### 5. Merge the dataframes

In [13]:
# despite different shapes both df have order_id which should match so we can do inner join for merge

In [14]:
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

### 6. Use indicator to check for full match

In [15]:
#Check for true match with value_counts()

In [16]:
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [17]:
#The inner join only pulls those that are included in both sets

In [18]:
pd.merge(df_ords, df_ords_prior, on = ['order_id'], how = 'outer')

,Unnamed: 0,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,0,2539329,1,1,2,8,median,196.0,1.0,0.0
1,0,2539329,1,1,2,8,median,14084.0,2.0,0.0
2,0,2539329,1,1,2,8,median,12427.0,3.0,0.0
3,0,2539329,1,1,2,8,median,26088.0,4.0,0.0
4,0,2539329,1,1,2,8,median,26405.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...
32640693,3421081,2977660,206209,13,1,12,7.0,38730.0,6.0,0.0
32640694,3421081,2977660,206209,13,1,12,7.0,31477.0,7.0,0.0
32640695,3421081,2977660,206209,13,1,12,7.0,6567.0,8.0,0.0
32640696,3421081,2977660,206209,13,1,12,7.0,22920.0,9.0,0.0


In [19]:
# Testing the outer merge shows 32,640,698 vs 32,434,489 rows of the inner merge
# This means we are missing 206,209 values that are not include in both, but we won't include that data

### Minor tweaking to rename dow again before exporting

In [20]:
df_merged_large.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

### Tweaking again to drop the unnamed: 0 column which came from exporting the index on orders

In [21]:
df_merge_large = df_merged_large.drop(columns = ['Unnamed: 0'])

In [22]:
## Double checking tweaks

In [23]:
df_merge_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,median,196,1,0,both
1,2539329,1,1,2,8,median,14084,2,0,both
2,2539329,1,1,2,8,median,12427,3,0,both
3,2539329,1,1,2,8,median,26088,4,0,both
4,2539329,1,1,2,8,median,26405,5,0,both


In [25]:
df_merge_large.shape

(32434489, 10)

### Export as Pickle File

In [26]:
df_merge_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))